In [ ]:
##check account balance
##check existing orders
##check tracking price
##check status
##if no orders and available balance:
### pull market prices
### input buy_order(tracking_price * (1-req))
### status = buy
###elif order and no available balance:
### if order is buy:
###     input sell_ordeR(tracking_price) * (1*req)
### else:
###.    status = sell
###else if no_order and no available_balance:
### db.send("fcked")
### sleep(5mins)

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
load_dotenv()
import os
bearer = os.getenv("TWITTERBEARER")

In [ ]:
https://api.twitter.com/2/tweets/counts/recent?query=ethereum

In [2]:
accounts = "https://api-public.sandbox.exchange.coinbase.com/accounts"
spot_prices = "https://api.exchange.coinbase.com/products/product_id/ticker"
orders = "https://api.exchange.coinbase.com/orders?sortedBy=created_at&sorting=desc&limit=100&status=%5B%27open%27%2C%20%27pending%27%5D"

headers = {"Accept": "application/json"}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"message":"Unauthorized."}


In [ ]:
import requests
### order example
url = "https://api.exchange.coinbase.com/orders"

payload = {
    "profile_id": "default profile_id",
    "type": "limit",
    "side": "buy",
    "stp": "dc",
    "stop": "loss",
    "time_in_force": "GTC",
    "cancel_after": "min",
    "post_only": "false"
}
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)